In [1]:
import napari
import os
import json
from skimage.io import imread, imshow
from octopuslite import DaskOctopusLiteLoader
from natsort import natsorted
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

# Load experiment information

To iteratively load each experiment (and label with experiment info?)

In [4]:
# define the scope
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
# add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('/home/nathan/gdrive_access_key.json', scope)
# authorize the clientsheet 
client = gspread.authorize(creds)
# load the cellx experiment directory worksheet
sheet = client.open('CellX Experiment Directory')
sheet_instance = sheet.get_worksheet(1)
# load the necessary columns 
expts = sheet_instance.col_values(col=1)
positions = sheet_instance.col_values(col=2)
condition = sheet_instance.col_values(col=3)
expt_notes = sheet_instance.col_values(col=5)
pos_notes = sheet_instance.col_values(col=6)
usability = sheet_instance.col_values(col=7)
# format the columns into dict then df
info = {'Experiments': expts, 'Positions':positions, 'Condition': condition, 'Position notes': pos_notes, 
        'Experiment Notes': expt_notes, 
        'Usable?':usability}
df = pd.DataFrame(info)
df = df.drop(index=df.index[0:2])
expt_status_df = df[~df['Experiments'].astype(str).str.startswith('ND')]
df = df[df['Experiments'].astype(str).str.startswith('ND')]
df = df.reset_index()
del df['index']

In [5]:
df

,Experiments,Positions,Condition,Position notes,Experiment Notes,Usable?
0,ND0000,Pos0,MDCK Rasv12 -,uninduced,stopped due to focus issue,FALSE
1,ND0000,Pos1,50:50 wt:ras+,induced,stopped due to focus issue,FALSE
2,ND0000,Pos2,MDCK Rasv12 +,induced,stopped due to focus issue,FALSE
3,ND0000,Pos3,50:50 wt:ras+,induced,stopped due to focus issue,FALSE
4,ND0000,Pos4,50:50 wt:ras-,uninduced,stopped due to focus issue,FALSE
...,...,...,...,...,...,...
336,ND0025,Pos9,97.5:2.5 wt:ras+,induced 3x seed dens,,TRUE
337,ND0025,Pos10,97.5:2.5 wt:ras+,induced 3x seed dens,,TRUE
338,ND0025,Pos11,97.5:2.5 wt:ras+,induced 3x seed dens,,TRUE
339,ND0025,Pos12,97.5:2.5 wt:ras+,induced 3x seed dens,,TRUE


In [6]:
df.to_csv('/home/nathan/data/kraken/ras/experiment_info_march22.csv')

In [6]:
df.loc[(df['Experiments'] == 'ND0013') & (df['Condition'] == "90:10 wt:ras+" )]

,Experiments,Positions,Condition,Position notes,Experiment Notes,Usable?
162,ND0013,Pos3,90:10 wt:ras+,induced,,TRUE
163,ND0013,Pos4,90:10 wt:ras+,induced,,TRUE
164,ND0013,Pos5,90:10 wt:ras+,induced,,TRUE
165,ND0013,Pos6,90:10 wt:ras+,induced,,TRUE
166,ND0013,Pos7,90:10 wt:ras+,induced,,TRUE
167,ND0013,Pos8,90:10 wt:ras+,induced,,TRUE
168,ND0013,Pos9,90:10 wt:ras+,induced,,TRUE
169,ND0013,Pos10,90:10 wt:ras+,induced,,TRUE
170,ND0013,Pos11,90:10 wt:ras+,"induced, wt +bleo",,TRUE


# Load images in Napari

From experiment info, load each set of images and launch napari if experimental condition is met

In [ ]:
root_dir = '/home/nathan/data/kraken/ras/'

viewer = napari.Viewer()
ratio = '90:10'
induced = True
tracks = False
aligned = True
#expts = set(df['Experiments'])
expts = ['ND0013']

for expt in expts:
    expt_dir = os.path.join(root_dir, expt)
    if os.path.exists(expt_dir):
        positions = natsorted([pos for pos in os.listdir(expt_dir) if pos.startswith('Pos') and os.path.isdir(os.path.join(expt_dir, pos))])
        for pos in positions:

            ### load expt condition
            condition = df['Condition'].loc[(df['Experiments'] == expt) & (df['Positions'] == pos)].values[0]
            ### if i want only induced experiments
            if induced:
                ### skip uninduced expts
                if '-' in condition:
                    print(expt, pos, condition, 'uninduced, skipping to next expt.')
                    continue
            ### check condition for correct ratio and skip if not
            if ratio not in condition:
                print(expt, pos, condition, 'incorrect seeding ratio, skipping to next expt.')
                continue
            ### load images
            else:
                print('Loading', expt, pos, condition)
                ### define image path
                image_path = os.path.join(root_dir, expt, pos, f'{pos}_images')
                ### check if alignment is to be applied
                if aligned:
                    transform_path = os.path.join(root_dir, expt, pos, 'gfp_transform_tensor.npy')
                else:
                    transform_path = False
                ### load images
                images = DaskOctopusLiteLoader(image_path, 
                                               transforms = transform_path,
                                               crop = (1200,1600))
                ### launch the napari viewer
                #viewer = napari.Viewer()
                viewer.add_image(images['gfp'], colormap= 'green', name= 'gfp', 
                                 blending = 'additive', contrast_limits = [0,150])
                viewer.add_image(images['rfp'], colormap= 'red', name='rfp', 
                                 blending = 'additive', contrast_limits = [0,32])
                viewer.add_image(images['irfp'], colormap= 'blue', name='irfp', 
                                 blending = 'additive')
                viewer.add_labels(images['mask'], name = f'{expt, pos} masks')
            
                ### check if tracks are to be added too
                if tracks:
                    tracks_path = ''
                    with btrack.dataio.HDF5FileHandler(tracks_path, 'r',obj_type=obj_type) as h:
                        tracks = h.tracks
                    ### need to configure this
                    data, properties, graph = btrack.utils.tracks_to_napari(tracks, ndim=2)
                    tracker_layer = viewer.add_tracks(data, properties=properties,graph=graph,name='tracks_1')
#                 points_layer = ...
                viewer.show(block=True)


# checking transforms

In [14]:
root_dir = '/home/nathan/data/kraken/ras/'

viewer = napari.Viewer()
ratio = '90:10'
induced = True
tracks = False
aligned = True
#expts = set(df['Experiments'])
expts = ['ND0013']

for expt in expts:
    expt_dir = os.path.join(root_dir, expt)
    if os.path.exists(expt_dir):
        
        positions = natsorted([pos for pos in os.listdir(expt_dir) if pos.startswith('Pos') and os.path.isdir(os.path.join(expt_dir, pos))])
        for pos in positions:

            ### load expt condition
            condition = df['Condition'].loc[(df['Experiments'] == expt) & (df['Positions'] == pos)].values[0]
            ### if i want only induced experiments
            if induced:
                ### skip uninduced expts
                if '-' in condition:
                    print(expt, pos, condition, 'uninduced, skipping to next expt.')
                    continue
            ### check condition for correct ratio and skip if not
            if ratio not in condition:
                print(expt, pos, condition, 'incorrect seeding ratio, skipping to next expt.')
                continue
            ### load images
            else:
                print('Loading', expt, pos, condition)
                ### define image path
                image_path = os.path.join(root_dir, expt, pos, f'{pos}_images')
                ### check if alignment is to be applied
                if aligned:
                    transform_path = os.path.join(root_dir, expt, pos, 'gfp_transform_tensor.npy')
                else:
                    transform_path = False
                ### load images
                images = DaskOctopusLiteLoader(image_path, 
                                               transforms = transform_path,
                                               crop = (1200,1600))
                ### launch the napari viewer
                #viewer = napari.Viewer()
                viewer.add_image(images['gfp'], colormap= 'green', name= 'gfp', 
                                 blending = 'additive', contrast_limits = [0,150])
                viewer.add_image(images['rfp'], colormap= 'red', name='rfp', 
                                 blending = 'additive', contrast_limits = [0,32])
                viewer.add_image(images['irfp'], colormap= 'blue', name='irfp', 
                                 blending = 'additive')
                viewer.add_labels(images['mask'], name = f'{expt, pos} masks')
            
                ### check if tracks are to be added too
                if tracks:
                    tracks_path = ''
                    with btrack.dataio.HDF5FileHandler(tracks_path, 'r',obj_type=obj_type) as h:
                        tracks = h.tracks
                    ### need to configure this
                    data, properties, graph = btrack.utils.tracks_to_napari(tracks, ndim=2)
                    tracker_layer = viewer.add_tracks(data, properties=properties,graph=graph,name='tracks_1')
#                 points_layer = ...
                viewer.show(block=True)


dask.array<astype, shape=(2788, 1200, 1600), dtype=uint8, chunksize=(1, 1200, 1600), chunktype=numpy.ndarray>

# Extrusion information extraction function
(on pause as I realised I need full track info for focal extrusion)

In [42]:
### load json file with events
event_dict_fn = '/home/nathan/data/kraken/scr/h2b/giulia/experiment_information/extrusions/extrusions.json'
with open(event_dict_fn, 'r') as file:
    event_dict = json.load(file)
    
### 

event_dict.append(event_ID)

{}

In [ ]:
@viewer.bind_key('w')
def note_event(event=None):
    if tracks:
        cell_ID = tracker_layer._get_value()
        position = tracker_layer.coordinates 
    else:
        ch = 'RFP'
        position = 
        cell_ID = 'na'
        points_layer.data = np.append(points_layer.data,cell_positions,axis=0)
    event_ID = f'{expt}_{pos}_{cell_ID}_{ch}':f'{int(position[0])}\n'
    with open(txt_address,'a') as txt_file:
        txt_file.write(txt_entry)
    txt_file.close()
    selected_track_lengths.append(len(cell_track.t))
    latest_added_track.append('r')
    

# @viewer.bind_key('w')
# def erase_latest_entry(event=None):
#     global selected_track_lengths 
#     global latest_added_track
#     with open(txt_address,'r+') as txt_file:
#         lines = txt_file.readlines()
#         txt_file.seek(0)
#         for line in lines[:-1]:
#             txt_file.write(line)
#         txt_file.truncate()
#     txt_file.close()
#     if latest_added_track[-1] == 's':
#         stitched_points_layer.data = stitched_points_layer.data[:-selected_track_lengths[-1]]
#     elif latest_added_track[-1] == 'r':
#         points_layer.data = points_layer.data[:-selected_track_lengths[-1]]
#     selected_track_lengths = selected_track_lengths[:-1]
#     latest_added_track = latest_added_track[:-1]
